In [1]:
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 경로 참 더럽네
from CRNNKerasmaster.parameter import *
from CRNNKerasmaster.Model import get_Model
from CRNNKerasmaster.Image_Generator import TextImageGenerator

import cv2
import itertools, os, time
import numpy as np
import argparse
import easydict
from datetime import datetime

K.set_learning_phase(0)

2022-12-19 16:31:28.487331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 16:31:28.828464: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 16:31:28.828541: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-19 16:31:29.982136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
def decode_label(out):
    # out : (1, 32, 42)
    out_best = list(np.argmax(out[0, 2:], axis=1))  # get max index -> len = 32
    out_best = [k for k, g in itertools.groupby(out_best)]  # remove overlap value
    outstr = ''
    for i in out_best:
        if i < len(letters):
            outstr += letters[i]
    return outstr

In [4]:
def label_to_text(label):  # eng -> hangul
    res = ''
    for i in label:
        res+=letters[i]
    return res

In [84]:
hdf=[i for i in os.listdir('./') if '.hdf5'in i]
hdf

['LSTM+BN5--2022_12_19_14_22_52.hdf5',
 'LSTM+BN5--2022_12_19_14_47_37.hdf5',
 'LSTM+BN5--text9--1000--102.089.hdf5',
 'LSTM+BN5--text9--1000--102.129.hdf5',
 'LSTM+BN5--text9--1000--104.370.hdf5',
 'LSTM+BN5--text9--1000--104.974.hdf5',
 'LSTM+BN5--text9--1000--111.037.hdf5',
 'LSTM+BN5--text9--1000--112.230.hdf5',
 'LSTM+BN5--text9--1000--119.030.hdf5',
 'LSTM+BN5--text9--1000--130.722.hdf5',
 'LSTM+BN5--text9--1000--137.869.hdf5',
 'LSTM+BN5--text9--1000--143.585.hdf5',
 'LSTM+BN5--text9--1000--68.436.hdf5',
 'LSTM+BN5--text9--1000--73.074.hdf5',
 'LSTM+BN5--text9--1000--74.051.hdf5',
 'LSTM+BN5--text9--1000--79.769.hdf5',
 'LSTM+BN5--text9--1000--85.715.hdf5',
 'LSTM+BN5--text9--1000--86.647.hdf5',
 'LSTM+BN5--text9--1000--87.225.hdf5',
 'LSTM+BN5--text9--1000--88.209.hdf5',
 'LSTM+BN5--text9--1000--89.023.hdf5',
 'LSTM+BN5--text9--1000--90.462.hdf5',
 'LSTM+BN5--text9--1000--91.454.hdf5',
 'LSTM+BN5--text9--1000--92.469.hdf5',
 'LSTM+BN5--text9--1000--93.856.hdf5',
 'LSTM+BN5--tex

In [89]:
w_file = hdf[13]
w_file

'LSTM+BN5--text9--1000--73.074.hdf5'

In [ ]:
res = []
for iii in range(13, len(hdf)-2):
    w_file = hdf[iii]
    exec(cmd)
    res.append([letter_acc / letter_total, iii,w_file])

...Previous weight data...
1/1 [==============================] - 1s 896ms/step
Predicted: 쪽868-등88380  /  True: 1588-1300
1/1 [==============================] - 0s 36ms/step
Predicted: T77273얘3532  /  True: T863-7737
1/1 [==============================] - 0s 33ms/step
Predicted: T4S7액4T듣T곳-툼-녕489  /  True: T864-1250
Time :  0.41390490531921387
ACC :  0.0
letter ACC :  0.1282051282051282


In [95]:
res

[[8, 'LSTM+BN5--text9--1000--73.074.hdf5', 0.1282051282051282],
 [8, 'LSTM+BN5--text9--1000--74.051.hdf5', 0.1282051282051282],
 [8, 'LSTM+BN5--text9--1000--79.769.hdf5', 0.075],
 [8, 'LSTM+BN5--text9--1000--85.715.hdf5', 0.13157894736842105],
 [8, 'LSTM+BN5--text9--1000--86.647.hdf5', 0.06382978723404255],
 [8, 'LSTM+BN5--text9--1000--87.225.hdf5', 0.1282051282051282],
 [8, 'LSTM+BN5--text9--1000--88.209.hdf5', 0.10204081632653061],
 [8, 'LSTM+BN5--text9--1000--89.023.hdf5', 0.1282051282051282],
 [8, 'LSTM+BN5--text9--1000--90.462.hdf5', 0.10416666666666667],
 [8, 'LSTM+BN5--text9--1000--91.454.hdf5', 0.11904761904761904],
 [8, 'LSTM+BN5--text9--1000--92.469.hdf5', 0.11904761904761904],
 [8, 'LSTM+BN5--text9--1000--93.856.hdf5', 0.16666666666666666],
 [8, 'LSTM+BN5--text9--1000--95.152.hdf5', 0.041666666666666664],
 [8, 'LSTM+BN5--text9--1000--95.195.hdf5', 0.0975609756097561],
 [8, 'LSTM+BN5--text9--1000--98.007.hdf5', 0.07142857142857142]]

In [93]:
cmd="""args = easydict.EasyDict({
    "weight":w_file,
    "test_img":"./CRNNKerasmaster/DB/test_9/"
})
# Get CRNN model
model = get_Model(training=False)

try:
    model.load_weights(args.weight)
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")
test_dir =args.test_img
test_imgs = os.listdir(args.test_img)
total = 0
acc = 0
letter_total = 0
letter_acc = 0
start = time.time()
for test_img in test_imgs:
    img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)

    img_pred = img.astype(np.float32)
    img_pred = cv2.resize(img_pred, (128, 64))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)

    net_out_value = model.predict(img_pred)

    pred_texts = decode_label(net_out_value)

    for i in range(min(len(pred_texts), len(test_img[0:-4]))):
        if pred_texts[i] == test_img[i]:
            letter_acc += 1
    letter_total += max(len(pred_texts), len(test_img[0:-4]))

    if pred_texts == test_img[0:-4]:
        acc += 1
    total += 1
    print('Predicted: %s  /  True: %s' % (pred_texts, test_img[0:-4]))
    
    # cv2.rectangle(img, (0,0), (150, 30), (0,0,0), -1)
    # cv2.putText(img, pred_texts, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)

    #cv2.imshow("q", img)
    #if cv2.waitKey(0) == 27:
    #   break
    #cv2.destroyAllWindows()

end = time.time()
total_time = (end - start)
print("Time : ",total_time / total)
print("ACC : ", acc / total)
print("letter ACC : ", letter_acc / letter_total)"""

In [62]:
args = easydict.EasyDict({
    "weight":w_file,
    "test_img":"./CRNNKerasmaster/DB/train_9/"
})

In [63]:
# Get CRNN model
model = get_Model(training=False)

try:
    model.load_weights(args.weight)
    print("...Previous weight data...")
except:
    raise Exception("No weight file!")

...Previous weight data...


In [64]:
test_dir =args.test_img
test_imgs = os.listdir(args.test_img)
total = 0
acc = 0
letter_total = 0
letter_acc = 0

In [65]:
start = time.time()
for test_img in test_imgs:
    img = cv2.imread(test_dir + test_img, cv2.IMREAD_GRAYSCALE)

    img_pred = img.astype(np.float32)
    img_pred = cv2.resize(img_pred, (128, 64))
    img_pred = (img_pred / 255.0) * 2.0 - 1.0
    img_pred = img_pred.T
    img_pred = np.expand_dims(img_pred, axis=-1)
    img_pred = np.expand_dims(img_pred, axis=0)

    net_out_value = model.predict(img_pred)

    pred_texts = decode_label(net_out_value)

    for i in range(min(len(pred_texts), len(test_img[0:-4]))):
        if pred_texts[i] == test_img[i]:
            letter_acc += 1
    letter_total += max(len(pred_texts), len(test_img[0:-4]))

    if pred_texts == test_img[0:-4]:
        acc += 1
    total += 1
    print('Predicted: %s  /  True: %s' % (pred_texts, test_img[0:-4]))
    
    # cv2.rectangle(img, (0,0), (150, 30), (0,0,0), -1)
    # cv2.putText(img, pred_texts, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)

    #cv2.imshow("q", img)
    #if cv2.waitKey(0) == 27:
    #   break
    #cv2.destroyAllWindows()

end = time.time()
total_time = (end - start)
print("Time : ",total_time / total)
print("ACC : ", acc / total)
print("letter ACC : ", letter_acc / letter_total)

1/1 [==============================] - 2s 2s/step
Predicted: 1588-3772  /  True: 1588-3772
1/1 [==============================] - 0s 157ms/step
Predicted: since1957  /  True: since1957
1/1 [==============================] - 0s 74ms/step
Predicted: Since2008  /  True: Since2008
1/1 [==============================] - 0s 69ms/step
Predicted: SKTelecom  /  True: SKTelecom
1/1 [==============================] - 0s 117ms/step
Predicted: T322-7569  /  True: T322-7569
1/1 [==============================] - 0s 83ms/step
Predicted: T472-2882  /  True: T472-2882
1/1 [==============================] - 0s 86ms/step
Predicted: T477-0113  /  True: T477-0113
1/1 [==============================] - 0s 76ms/step
Predicted: T486-3176  /  True: T486-3176
1/1 [==============================] - 0s 60ms/step
Predicted: T486-5533  /  True: T486-5533
1/1 [==============================] - 0s 33ms/step
Predicted: T486-5535  /  True: T486-5535
1/1 [==============================] - 0s 39ms/step
Predicted: T489-14